In [193]:
import ct.sm as sm
import cantera as ct
import scipy.constants as C
import matplotlib.pyplot as plt
# import CoolProp.CoolProp as cp
import importlib
# import sm
import numpy as np
# import pint
importlib.reload(sm)

<module 'ct.sm' from 'e:\\Code\\autism\\ss13\\supermattercore\\ct\\sm.py'>

In [194]:
new_he = sm.HeatExchanger(191,0,0)
s = ct.Solution('./ct/xgm.yaml')
nu_N2 = sm.CANISTER_VOLUME*sm.ENG_CANISTER_PRESSURE/8.314/293
cool_V = .77 + 1.47 + 13.37

In [195]:
s.TPX = 300, nu_N2*8.314*300/cool_V, 'N2: 1'
q = ct.Quantity(s, moles=nu_N2/1e3)
q.volume

15.610868591456823

In [196]:
q2 = ct.Quantity(s, moles=2)
q2.volume

8.340183043677058

In [197]:
q.constant = q2.constant = 'TV'
q3 = q + q2
q3.volume

23.95105163513388

In [198]:
q.T

300.0

improved mathmodel soln attempt

In [199]:
#constants
Vc1 = .67 #cool turbine inp
Vc2 = 13.37+1.47+0.77 + 3*.2 # this is wat is cooled/ cool turb out
Vh2 = 1.995 + .2 +.7 + 2 # filters / hot turb out
Vh1 = .315 + .4 + .2 # hot turb in
Vh0 = 37.5 # room

N_space = 191

R = 8.314
cv = 20

eta_T = .65 # TEG

nu_canister = 3743 # not precise

AIR_PCRIT = 3771e3 # Pa
AIR_TCRIT = 132.65 # K, 
AIR_VMOLAR = R * AIR_TCRIT / AIR_PCRIT

#parameters
Eer = 112
N_hot=N_cold=3

In [200]:
def est_dTcooled(T: float, n: int, nu: float):
    Td = T - 3.15 #-TCMB
    Cve = cv*max(Vc1/AIR_VMOLAR, nu)
    k1 = 3*n*C.sigma*2/Cve
    k2 = n*0.08*200/Cve
    return Td*(1 - (k1*Td**3+1)**(-1/3))-k2

def est_turbine(Vistar, Pi, Ti, Po):
    eta_K, kappa, Vi, fv = .04, .66, .4, .2
    eta_V = eta_K/kappa * (1-fv**kappa)
    dP = max(Pi-Po-5e3, 0)
    if dP == 0: return 0, 0
    dnu_max = dP * Vistar / 3 / Ti / R
    Ei = min(dP*Vistar, Pi*Vi)
    E = Ei * eta_V
    nu = min(dnu_max, Pi*Vi/R/Ti)
    return nu, Ei

def est_S(T, P):
    if P==0: return 15e3
    c1 = (0.028*cv)**(2/3)*R
    return R*(15 + np.log(
        1 + c1*T**(2/3)/P
    ))

def est_pump(Vi, Vo, Nmax, Pi, Ti, Po, To, Vostar):
    Pstar = 15e6
    dP = Pstar - Po
    if dP < 100: return 0,0
    nu_in = Pi*Vi/R/Ti
    nu_a = dP*Vostar/R/To
    Co = cv*Po*Vo/R/To
    C1 = cv*nu_a
    T1 = (To*Co+Ti*C1)/(Co+C1)
    nu_tgt = dP*Vostar/R/T1
    nu_all = min(Pi*Vi/R/Ti,nu_tgt)
    S_spec = est_S(To,Po) - est_S(Ti,Pi)
    N_s = 0
    if S_spec < 0:
        Ns = -S_spec*To/2.5
        nu_all = min(nu_all, Nmax/Ns)
    return nu_all, N_s*nu_all

In [201]:
est_pump(Vh0, .2, 30e3, 102e5,5e3,153e5,5e3,Vh1)

(0, 0)

In [202]:
def try_soln(x, showE = False):
    NTh = Eer * 5.5e3
    fh0,fh1,fc1,Tc,Th,Tw = x
    
    # normalize mass fractions
    if fc1>1 or fc1<0: fc1=abs(fc1/(1+fc1**2))
    if fh0>1 or fh0<0: fh0=abs(fh0/(1+fh0**2))
    if fh1>1 or fh1<0: fh1=abs(fh1/(1+fh1**2))
    if (fh0+fh1)>1: fh0=fh1=fh0+fh1/(1+fh0+fh1)/2

    nu_h = N_hot*nu_canister
    nu_c = N_cold*nu_canister
    # nu_h = ((Ph0*Vh0+Ph1*Vh1)/Th+Ph2*Vh2/Tw)/R
    nu_h0 = nu_h*fh0
    nu_h1 = nu_h*fh1
    nu_h2 = nu_h*(1-fh0-fh1)
    Ph0 = nu_h0 *R*Th/Vh0
    Ph1 = nu_h1 *R*Th/Vh1
    Ph2 = nu_h2 *R*Tw/Vh2
    # nu_c = nu_c1 + (Pc2*Vc2)/R/Tc
    nu_c1 = nu_c*fc1
    nu_c2 = nu_c*(1-fc1)
    Pc1 = nu_c1 *R*Tc/Vc1
    Pc2 = nu_c2 *R*Tc/Vc2
    if showE:
        print(f'{nu_h0*.028:.3g} kg @{Ph0/1e5:.3g} bar in room, {nu_h1*.028:.3g} kg @{Ph1/1e5:.3g} bar  in scalding, {nu_h2*.028:.3g} kg  @{Ph2/1e5:.3g} bar  in warm')
        print(f'{nu_c1*.028:.3g} kg @{Pc1/1e5:.3g} bar in smol, {nu_c2*.028:.3g} kg @{Pc2/1e5:.3g} bar in cooling w/rad')

    dT_radiat = est_dTcooled(Tc, N_space, nu_c2)
    if showE: print('radiated',dT_radiat,'K')

    dnu_h, E1 = est_turbine(Vh1, Ph1, Th, Ph2) # nu T1
    dnu_c, E2 = est_turbine(Vc1, Pc1, Tc, Pc2) # nu T2
    if showE: 
        print('T1 pumped',dnu_h,'mole and stored',E1/1e3, 'kJ')
        print('T2 pumped',dnu_c,'mole and stored',E2/1e3, 'kJ')
    
    dnu_HP1, E_HP1 = est_pump(.2, .2,  45e3, Pc2,Tc,Pc1,Tc,Vc1) # HP1
    dnu_PI1, E_PI1 = est_pump(Vh0, .2, 30e3, Ph0,Th,Ph1,Th,Vh1) # PI1 -- ?? check
    dnu_PO1, E_PO1 = est_pump(.7, Vh0, 45e3, Ph2,Tw,Ph0,Th,Vh0) # PO1
    if showE:
        print('HP1',dnu_HP1,'PI1',dnu_PI1,'PO1',dnu_PO1)
    dT_TEG = Th - Tc
    tilde_C = cv * dnu_h * dnu_c / (dnu_h+dnu_c) if (dnu_h+dnu_c)!=0 else 0
    dE_TEG = dT_TEG * tilde_C

    if showE:
        print ('TEG exchd: ', dE_TEG/1e3, 'kJ')
        print ('Produced:',(dE_TEG*eta_T + E1 + E2)/1e3,'kW')
        print ('Pumps ate:',[E_HP1, E_PI1, E_PO1])


    return [
        # mass balance  
        (dnu_PO1 - dnu_PI1)/nu_h, #fh0 change
        (dnu_PI1 - dnu_h)/nu_h, #fh1 change
        (dnu_HP1 - dnu_c)/nu_c, # fc1 change
        # temp balance   --> Tc, Th, Tw
        # (1-eta_T)*NTh/cv/nu_c - dT_radiat, # Tc rough
        (1-eta_T)*dE_TEG/cv/nu_c - dT_radiat, # Tc rough
        (NTh - dE_TEG)/cv/nu_h, #Th rough 
        ((Th - Tw)*cv*dnu_h - dE_TEG)/cv/nu_h2, #Tw rough
        # heat balance (1)
    ]

In [203]:
x0 = np.array([0.14,0.143,0.02, 300, 1000, 580])

In [210]:
for i in range(1000):
    dx1 = try_soln(x0)
    x0 += np.array(dx1)*.5
dx1 = try_soln(x0, True)
print('Temps', x0[-3:])
print('dmfs', dx1[:3])
print('dTs', dx1[-3:])

152 kg @7.39 bar in room, 60.8 kg @121 bar  in scalding, 102 kg  @34.2 bar  in warm
22.9 kg @31.9 bar in smol, 292 kg @16.8 bar in cooling w/rad
radiated 0.9600142516257577 K
T1 pumped 518.8979425932883 mole and stored 4842.253437342026 kJ
T2 pumped 128.46361429542534 mole and stored 1008.5529675585584 kJ
HP1 128.46361429543023 PI1 518.897942608967 PO1 518.8979425670375
TEG exchd:  616.0000009841368 kJ
Produced: 6251.2064055402725 kW
Pumps ate: [0.0, 0.0, 0.0]
Temps [314.76562669 613.87879798 554.52223186]
dmfs [-3.734039072146722e-12, 1.3962655158095812e-12, 4.353490089971938e-16]
dTs [-1.2719957309670349e-09, -4.382121427037251e-09, -3.57543161677653e-09]


In [43]:
from scipy.optimize import root

In [95]:
root(try_soln, [.82,.03,.05, 300, 5000, 2800], options={'maxfev': 5000})

C:\Users\emera\AppData\Local\Temp\ipykernel_3020\1742409440.py:23: RuntimeWarning: invalid value encountered in scalar power
  1 + c1*T**(2/3)/P


 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 1.460e-01 -1.212e-01  1.248e-07 -1.816e-02 -3.320e+00
            0.000e+00]
       x: [ 7.164e-01  1.098e-01  7.276e-02  3.162e+02 -1.292e+04
           -1.883e+03]
  method: hybr
    nfev: 24
    fjac: [[-5.315e-03 -4.997e-05 ... -9.541e-02  9.954e-01]
           [ 3.315e-02  3.093e-03 ... -9.949e-01 -9.518e-02]
           ...
           [-7.065e-01  7.073e-01 ... -2.080e-02 -5.731e-03]
           [ 7.069e-01  7.069e-01 ...  2.516e-02  6.221e-03]]
       r: [ 3.038e+01 -2.459e+03 ...  2.553e-06  4.128e-14]
     qtf: [ 3.160e-01  3.307e+00  1.736e-02  5.946e-03 -1.199e-01
           -6.596e-02]

In [47]:
try_soln([1.5e7,1.4e7,2.6e6,4.8e2,9.8e4,7.3e2], True)

19.3 kg in room, 0.44 kg in scalding, 295 kg in warm
296 kg in smol postpump, 18.7 kg in cooling w/rad
radiated 1.0032823798073216 K
T1 pumped 0.3548161570048222 mole and stored 867.282809625399 kJ
T2 pumped 0 mole and stored 0.0 kJ
HP1 199.40253028817912 PI1 1.1230135547122386 PO1 0.0
Produced: 867.282809625399 kW
Pumps ate: [0.0, 0.0, 0.0]


[-199.40253028817912,
 -0.7681973977074164,
 0.3548161570048222,
 690259.3518371811,
 -616000.0,
 -3686.665166389139]